## Dataset import

Since we will focus on RNN in this notebooks I will jump to a pretrained vector dataset, this data set includes as X input embeddings representation of movies reviews and target a sentiment analysis label, you can learn more about the dataset here https://ai.stanford.edu/~amaas/data/sentiment/

In [ ]:
from tensorflow.keras.datasets import imdb

In [ ]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

In [ ]:
X_train

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]),
       list([1, 194, 1153, 194, 2, 78, 228, 5, 6, 1463, 4369,

In [ ]:
y_train

array([1, 0, 0, ..., 0, 1, 0])

## LSTM

In this first model we will use a many to one LSTM architecture, given some sequence context we will try to predict a sentiment first of all we will import tenssorflow and keras libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
tf.random.set_seed(7)

Now i will pad my sequence to have the same lenght for modeling, this is for learning porpuses.

In [ ]:
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

Now, we will define our LSTM and train the dataset

In [ ]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 32)           160000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               53200     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 213301 (833.21 KB)
Trainable params: 213301 (833.21 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/3
391/391 [==============================] - 453s 1s/step - loss: 0.4535 - accuracy: 0.7795 - val_loss: 0.3305 - val_accuracy: 0.8667
Epoch 2/3
391/391 [==============================] - 437s 1s/step - loss: 0.2874 - accuracy: 0.8849 - val_loss: 0.3334 - val_accura

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
scores

[0.33440855145454407, 0.8715999722480774]

##  LSTM with dropout

LSTM and RNN in general are very sensitive to overfit, so one simple mechanism to avoud is adding a drop out layer

In [ ]:
from tensorflow.keras.layers import Dropout

embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 32)           160000    
                                                                 
 dropout (Dropout)           (None, 500, 32)           0         
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 213301 (833.21 KB)
Trainable params: 213301 (833.21 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/3

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
scores

[0.3056398332118988, 0.877560019493103]

## BI **LSTM**

In [ ]:
from tensorflow.keras.layers import Bidirectional

embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 32)           160000    
                                                                 
 bidirectional (Bidirection  (None, 200)               106400    
 al)                                                             
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 266601 (1.02 MB)
Trainable params: 266601 (1.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/3
391/391 [==============================] - 969s 2s/step - loss: 0.6121 - accuracy: 0.6998
Epoch 2/3
391/391 [==============================] - 964s 2s/step - loss: 0.3391 - accuracy: 0.8585
Epoch 3/3
3

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
scores

[0.31282493472099304, 0.8697999715805054]

## BI-GRU

In [20]:
from tensorflow.keras.layers import GRU

embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Bidirectional(GRU(100, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 32)           160000    
                                                                 
 bidirectional_1 (Bidirecti  (None, 200)               80400     
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                                 
Total params: 240601 (939.85 KB)
Trainable params: 240601 (939.85 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/3
391/391 [==============================] - 819s 2s/step - loss: 0.4899 - accuracy: 0.7519
Epoch 2/3
391/391 [==============================] - 825s 2s/step - loss: 0.2909 - accuracy: 0.8818
Epoch 3

In [21]:
scores = model.evaluate(X_test, y_test, verbose=0)
scores

[0.29217344522476196, 0.8804399967193604]